In [1]:
import pandas as pd

In [2]:
import spacy
spacy.load('fr_core_news_sm')
from spacy.lang.fr import French
parser = French()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [3]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to /home/hamza/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
fr_stop = set(nltk.corpus.stopwords.words('french'))

In [5]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in fr_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [6]:
df_covid = pd.read_csv('/home/hamza/3A_projets/filiere/saved_data/df_french_clean.csv')

In [7]:
df_covid

,lang,text,username,search_url,datetime,replies,retweets,likes,is_share
0,fr,Ça continue 400 masques volés dans un service ...,SBurtey,https://twitter.com/search?q=epidemie%20until%...,2020-03-04 10:45:20,79,1265,3071,False
1,fr,La France : a atteint le stade 3 de l ' épidé...,RayanSmd,https://twitter.com/search?q=epidemie%20until%...,2020-03-04 17:01:40,16,420,2063,False
2,fr,On est chez les fous ! Le 17 fev Macron affr...,valerieboyer13,https://twitter.com/search?q=epidemie%20until%...,2020-03-04 20:32:25,107,933,1061,False
3,fr,"Avant, on avait des foyers d ' épidémie . Mai...",Aramisunchained,https://twitter.com/search?q=epidemie%20until%...,2020-03-04 22:47:23,37,191,590,False
4,fr,ça mbute yen a ils ont vraiment attendu une ép...,marinelbars,https://twitter.com/search?q=epidemie%20until%...,2020-03-04 20:16:01,6,1618,5347,False
...,...,...,...,...,...,...,...,...,...
12320,fr,EPIDEMIE - Il y a actuellement 999 cas de coro...,Conflits_FR,https://twitter.com/search?q=epidemie%20until%...,2020-02-18 18:09:49,28,56,194,False
12321,fr,Le bilan de l ' épidémie de Covid-19 dépasse l...,F24AsiePacifiq,https://twitter.com/search?q=epidemie%20until%...,2020-02-18 01:35:04,0,0,0,False
12322,fr,Voilà ce qu ' aurait dû dire Buzin ! Ça prena...,DONTOMASIN0,https://twitter.com/search?q=epidemie%20until%...,2020-02-18 01:34:14,1,0,4,False
12323,fr,"Non non, tu vas juste te manger des commentair...",DONTOMASIN0,https://twitter.com/search?q=epidemie%20until%...,2020-02-18 01:31:30,0,0,0,False


In [8]:
text_data = []
for index,rows in df_covid.iterrows():
    text = rows['text']
    tokens = prepare_text_for_lda(text)
    text_data.append(tokens)
    
    

In [9]:

from gensim import corpora
import pickle

dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [10]:
import gensim
NUM_TOPICS = 7
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')


In [40]:
topics = ldamodel.print_topics(num_words=20)
for topic in topics:
    print(topic)

(0, '0.042*"épidémie" + 0.036*"coronavirus" + 0.023*"chine" + 0.021*"morts" + 0.020*"personnes" + 0.019*"france" + 0.010*"décès" + 0.008*"million" + 0.008*"virus" + 0.008*"covid19france" + 0.007*"quarantaine" + 0.007*"nombre" + 0.007*"monde" + 0.007*"depuis" + 0.006*"wuhan" + 0.006*"covid19" + 0.005*"bilan" + 0.005*"chinois" + 0.004*"nouveaux" + 0.004*"santé"')
(1, '0.053*"coronavirus" + 0.036*"épidémie" + 0.033*"covid19france" + 0.029*"coronavirusfr" + 0.018*"covid19" + 0.015*"coronavirusfrance" + 0.011*"france" + 0.009*"covidー19" + 0.009*"chine" + 0.008*"covid_19" + 0.007*"covid19fr" + 0.007*"santé" + 0.006*"coronavirusenfrance" + 0.006*"gouvernement" + 0.006*"covid2019" + 0.005*"coronavid19" + 0.005*"macron" + 0.005*"paris" + 0.005*"virus" + 0.005*"stade"')
(2, '0.019*"épidémie" + 0.018*"masque" + 0.009*"contre" + 0.008*"coronavirus" + 0.008*"pandémie" + 0.007*"buzyn" + 0.006*"pleine" + 0.004*"santé" + 0.004*"covid19france" + 0.004*"cette" + 0.004*"protéger" + 0.003*"hôpitaux" + 0.0

In [11]:
path='/home/hamza/3A_projets/filiere/saved_data/df_covid_emotions.pkl'
df_emotions = pickle.load(open(path,'rb'))
emotions = ["N", "VALORISATION",
"DEVALORISATION",
"PEUR",
"DESACCORD",
"COLERE",
"ACCORD",
"MEPRIS"]
dict_map = dict(zip(range(len(emotions)),emotions))
df_emotions['emotions'] = df_emotions['predictions'].map(dict_map)


In [15]:
from collections import defaultdict
import numpy as np
d=defaultdict(int)
for index,rows in df_emotions[df_emotions.emotions=='PEUR'].iterrows():
    new_doc = prepare_text_for_lda(rows['text'])
    new_doc_bow = dictionary.doc2bow(new_doc)
    output = ldamodel.get_document_topics(new_doc_bow)
    output = np.array([i[1] for i in output])

    d[np.argmax(output)]+=1
dict(d)

{2: 112, 0: 227, 6: 108, 1: 121, 4: 99, 5: 21, 3: 61}

In [20]:
from collections import defaultdict
import numpy as np

for index,rows in df_emotions[df_emotions.emotions=='PEUR'].iterrows():
    new_doc = prepare_text_for_lda(rows['text'])
    new_doc_bow = dictionary.doc2bow(new_doc)
    output = ldamodel.get_document_topics(new_doc_bow)
    output = np.array([i[1] for i in output])
    if np.argmax(output) == 0:
        print(rows['text'])
        print('/n')
    #d[np.argmax(output)]+=1
dict(d)

AVIS A LA POPULATION En plus de l epidemie de COVID19fr le pays serait touché par une epidemie de Thiagoenterite
/n
Aujourd ' hui coronavirus devient l ' épidémie meurtrière qui inquiète le monde .  Coronavirus vient de dépasser 3000 morts pour 90 . 000 infections dans soixantaine de pays .  OMS vient de mettre 12 milliards de dollars pour la prévention dans différents pays .  La santé à tous .  Consel
/n
AS | Le gouvernement espagnol par le biais du CSD  ( Conseil Supérieur des Sports )  a réuni les fédérations et les ligues et a annoncé qu ' il étudiera la situation  ' cas par cas '  et que les matchs à risques se joueront à huit clos face à l ' épidémie du coronavirus  .
/n
La diffusion du virus COVID-19 commence à se propager à l ' ensemble de la planète et affecter les marchés financiers .  Dès lors, doit-on s ' inquiéter à long terme des répercussions économiques et sociétales d ' une telle épidémie  ?
/n
[  ALERTE INFO  ]  Le point sur l ' épidémie de coronavirus dans le monde  

{2: 112, 0: 227, 6: 108, 1: 121, 4: 99, 5: 21, 3: 61}